In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import utils
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import datetime
import numpy as np
import json




class CustomDataset(Dataset):
    def __init__(self, features, targets,model_type):
        self.features = torch.tensor(features, dtype=torch.float32)
        if model_type == 'classifier':
            self.targets = torch.tensor(targets, dtype=torch.long)
        elif model_type == 'regressor':
            self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]


def scale_column(x_train:pd.DataFrame, x_test:pd.DataFrame, column:list):
    """
    Move to utils_NN.py
    Scales the same nature columns of x_train and x_test using the MinMaxScaler. The reference column is the one with the maximum value.


    """
    max_value = x_train[column].max().max()
    x_train[column] = x_train[column]/max_value
    x_test[column]  = x_test[column]/max_value
    return x_train, x_test, max_value

pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Parameters

model_type =  'regressor' # 'classifier' or 'regressor'
use_trap_info = True
ntraps = 3
lags = 3
random_split = True
test_size = 0.2
scale = False
learning_rate =1e-3
batch_size = 64
epochs = 10



parameters = {
    'model_type': model_type,
'use_trap_info': use_trap_info,
'ntraps': ntraps,
'lags': lags,
'random_split': random_split,
'test_size': test_size,
'scale': scale,
'learning_rate': learning_rate,
'batch_size': batch_size,
'epochs': epochs   
}



In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
# data import and preprocessing
data = pd.read_csv(f'./results/final_df_lag{lags}_ntraps{ntraps}.csv')
n = data.shape[0]
nplaca_index = data['nplaca']
data.drop(columns=['nplaca','distance0'], inplace=True) # drop distance0 because it is always zero
ovos_flag = data['novos'].apply(lambda x: 1 if x > 0 else 0)

# divide columns into groups
days_columns = [f'days{i}_lag{j}' for i in range(ntraps) for j in range(1, lags+1)]
distance_columns = [f'distance{i}' for i in range(1,ntraps)]
eggs_columns = [f'trap{i}_lag{j}' for i in range(ntraps) for j in range(1, lags+1)]


In [5]:
# definition of x and y
if model_type == 'classifier':
    y = ovos_flag
elif model_type == 'regressor':
    y = data['novos']

if use_trap_info:
    x = data.drop(columns=['novos'])
else:
    drop_cols = ['novos'] + days_columns + distance_columns
    x = data.drop(columns=drop_cols)

In [6]:
# train test split
train_size = 1 - test_size

if random_split:
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42, stratify=ovos_flag)
else:
    y_train = y.iloc[:int(n*train_size)]
    y_test = y.iloc[int(n*train_size):]
    x_train = x.iloc[:int(n*train_size)]
    x_test = x.iloc[int(n*train_size):]    

In [7]:
# scaling
if scale:
    x_train, x_test, max_eggs = scale_column(x_train, x_test, eggs_columns)
    if use_trap_info:
        x_train, x_test, max_distance = scale_column(x_train, x_test, distance_columns)
        x_train, x_test, max_days = scale_column(x_train, x_test, days_columns)


    if model_type != 'classifier':
        y_train = y_train/max_eggs
        y_test = y_test/max_eggs

In [8]:
# transform to tensors
xtrain = torch.tensor(x_train.values, dtype=torch.float32).to(device)
xtest = torch.tensor(x_test.values, dtype=torch.float32).to(device)
if model_type == 'classifier':
    ytrain = torch.tensor(y_train.values, dtype=torch.long).to(device)
    ytest = torch.tensor(y_test.values, dtype=torch.long).to(device)
elif model_type == 'regressor':
    ytrain = torch.tensor(y_train.values, dtype=torch.float32).to(device)
    ytest = torch.tensor(y_test.values, dtype=torch.float32).to(device)
    
train_dataset = CustomDataset(xtrain, ytrain,model_type)
test_dataset = CustomDataset(xtest, ytest,model_type)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=random_split)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=random_split)

C:\Users\User\AppData\Local\Temp\ipykernel_16112\3919755799.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.float32)
C:\Users\User\AppData\Local\Temp\ipykernel_16112\3919755799.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(targets, dtype=torch.float32)


In [9]:
# Network structure
if use_trap_info:
    model_input = lags*ntraps + ntraps-1 + ntraps*lags # sum  of eggs, distances minus one and days
else:
    model_input = lags*ntraps
    
if model_type == 'classifier':
    model_output = 2
elif model_type == 'regressor':
    model_output = 1


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.structure = nn.Sequential(
            nn.Linear(model_input, 20),
            nn.Linear(20, 10),
            nn.Linear(10, 5),
            nn.Linear(5, model_output)

        )

    def forward(self, x):
        logits = self.structure(x)
        return logits    

In [38]:
# Create train and test loops
def train_loop(dataloader, model, loss_fn, optimizer,batch_size):
    size = xtrain.shape[0]
    model.train()
    for batch, (xtest, ytest) in enumerate(dataloader):
        # Compute prediction and loss
        optimizer.zero_grad()
        pred = model(xtest)
        loss = loss_fn(pred, ytest)
        # Backpropagation
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(xtest)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn,model_type):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, acc = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            
            if model_type == 'classifier':
                pred = model(X)
                acc += (pred.argmax(1) == y).type(torch.float).sum().item()
            elif model_type == 'regressor':
                pred = torch.round(model(X))
                acc += ((pred.round() == y).type(torch.float)).sum().item()

            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    acc /= size
    print(f"Test Metrics: \n Accuracy: {(100*acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return acc, test_loss

In [39]:
model = NeuralNetwork().to(device)

if model_type == 'classifier':
    loss_fn = nn.CrossEntropyLoss()
elif model_type == 'regressor':
    loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


loss_hist = []
accuracy_hist = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, batch_size)
    acc, test_loss = test_loop(test_dataloader, model, loss_fn,model_type)
    
    accuracy_hist.append(acc)
    loss_hist.append(test_loss)
    torch.save(model.state_dict(), f'./results/NN/save_parameters/model{model_type}_lags{lags}_ntraps{3}_epoch{t}.pth')
    
    
print("Done!")
utils.play_ending_song()
utils.stop_ending_song(2)

Epoch 1
-------------------------------
loss: 4653.574219  [   64/267656]


c:\Users\User\miniconda3\envs\venv_ovitraps\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 2225.621338  [ 6464/267656]
loss: 4686.448242  [12864/267656]
loss: 4635.913086  [19264/267656]
loss: 3015.839355  [25664/267656]
loss: 3711.317871  [32064/267656]
loss: 60772.812500  [38464/267656]
loss: 2462.866211  [44864/267656]
loss: 3956.755615  [51264/267656]
loss: 4601.730957  [57664/267656]
loss: 10129.496094  [64064/267656]
loss: 11949.902344  [70464/267656]
loss: 4189.386719  [76864/267656]
loss: 2557.769775  [83264/267656]
loss: 4199.195312  [89664/267656]
loss: 6696.571777  [96064/267656]
loss: 7039.033203  [102464/267656]
loss: 1585.639893  [108864/267656]
loss: 3773.840332  [115264/267656]
loss: 4470.018555  [121664/267656]
loss: 2304.821777  [128064/267656]
loss: 1234.786621  [134464/267656]
loss: 4509.984375  [140864/267656]
loss: 2602.822021  [147264/267656]
loss: 6723.731445  [153664/267656]
loss: 3756.863037  [160064/267656]
loss: 1067.533691  [166464/267656]
loss: 3017.060059  [172864/267656]
loss: 2888.207764  [179264/267656]
loss: 34433.941406  [185664/2676

c:\Users\User\miniconda3\envs\venv_ovitraps\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\User\miniconda3\envs\venv_ovitraps\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([34])) that is different to the input size (torch.Size([34, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Metrics: 
 Accuracy: 31.7%, Avg loss: 41943.466846 

Done!


In [21]:
if model_type == 'classifier':
    yhat = model(xtest).argmax(1).cpu().numpy()
elif model_type == 'regressor':
    yhat = model(xtest).round().cpu().detach().numpy() 

if model_type == 'classifier':
    print(accuracy_score(y_test, yhat))
    print(confusion_matrix(y_test, yhat, normalize='true', labels=[0,1]))


In [22]:
structure_path = f'./results/NN/save_structure/model{model_type}_lags{lags}_ntraps{3}_structure.pth'
torch.save(model, structure_path)

new_results = {
    'model': model_type,
    'net_structure': structure_path,
    'repetition': 1,
    'parameters': parameters,
    'date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'accuracy': accuracy_hist,
    'loss': loss_hist,
    'yhat': yhat.tolist(),
    'ytest': ytest.cpu().numpy().tolist(),
}


# Load the existing JSON file
with open('./results/NN/model_accuracies.json', 'r') as f:
    results = json.load(f)
    # Update with new results


for item in results:
    if item['parameters'] == new_results['parameters'] and item['net_structure'] == new_results['net_structure']:
        new_results['repetition'] = item['repetition'] + 1
results.append(new_results)



with open('./results/NN/model_accuracies.json', 'w') as file:
    json.dump(results, file, indent=4)

In [3]:

# Load the existing JSON file
with open('./results/NN/model_accuracies.json', 'r') as f:
    results = json.load(f)
    # Update with new results


In [13]:
model(xtest).round() == ytest

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])